## LSTM sample

In [ ]:
symbol = "AAP"

### imports

In [ ]:
from datetime import datetime, date, timedelta
import alpaca_trade_api as tradeapi
from typing import List, Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from fastai.data.all import *
import matplotlib.pyplot as plt

### Load data

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
ohlc_data = api.polygon.historic_agg_v2(
    symbol,
    1,
    "minute",
    _from=str(date.today() - timedelta(days=7)),
    to=str(date.today()),
).df

In [ ]:
x_dates = ohlc_data.index
ohlc_data['date'] = np.arange(start = 0, stop = len(ohlc_data), step = 1, dtype='int')
ohlc_data=ohlc_data.set_index(ohlc_data.date)

In [ ]:
ohlc_data.close.plot()
plt.show()

### Pre-processing

In [ ]:
ohlc_data["delta"] = ohlc_data.close.diff()
ohlc_data["delta_pct"] = ohlc_data.close.pct_change()

In [ ]:
ohlc_data.delta.plot()
plt.show()
ohlc_data.delta_pct.plot()
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ohlc_data.delta = scaler.fit_transform(ohlc_data.delta.values.reshape(-1, 1)).flatten()

plt.figure(figsize=(10, 4))
ohlc_data.delta.plot()

In [ ]:
ohlc_data.shape

## Training

In [ ]:
test_data_size = 100

train_data = ohlc_data.delta[2:-test_data_size]
test_data = ohlc_data.delta[-test_data_size:]
print(len(train_data))
print(len(test_data))

In [ ]:
test_data=pd.Series(test_data.tolist())

In [ ]:
train_data=pd.Series(train_data.tolist())

In [ ]:
train_data_normalized = torch.FloatTensor(train_data).view(-1)

In [ ]:
train_data_normalized

In [ ]:
train_window = 20
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)
train_inout_seq[:5]

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
print(model)

In [ ]:
epochs = 150

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

## Predicting